In [1]:
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
import numpy as np
import os
from transformers import ClapProcessor, ClapModel, AutoProcessor, ClapAudioModel, AutoFeatureExtractor


/home/matteoc/miniconda3/envs/huggin/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
device = 'cuda:1'
pipe = pipe.to(device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]/home/matteoc/miniconda3/envs/huggin/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Loading pipeline components...: 100%|██████████| 6/6 [00:06<00:00,  1.01s/it]


In [3]:
pipe.vae.config

FrozenDict([('encoder_hidden_size', 128),
            ('downsampling_ratios', [2, 4, 4, 8, 8]),
            ('channel_multiples', [1, 2, 4, 8, 16]),
            ('decoder_channels', 128),
            ('decoder_input_channels', 64),
            ('audio_channels', 2),
            ('sampling_rate', 44100),
            ('_class_name', 'AutoencoderOobleck'),
            ('_diffusers_version', '0.30.0.dev0'),
            ('_name_or_path',
             '/home/matteoc/.cache/huggingface/hub/models--stabilityai--stable-audio-open-1.0/snapshots/67351841772475731424cf358c9fb785716ac78f/vae')])

In [4]:
prompt = "The sound of a hammer hitting a wooden surface."
negative_prompt = "Low quality."
generator = torch.Generator((device)).manual_seed(0)
do_classifier_free_guidance=7.0
negative_prompt_embeds=None
audio_start_in_s=0
audio_end_in_s=10.0


prompt_embeds = pipe.encode_prompt(
            prompt,
            device=device,
            do_classifier_free_guidance=do_classifier_free_guidance,
            negative_prompt=negative_prompt,
            prompt_embeds=None,
            negative_prompt_embeds=negative_prompt_embeds,
            attention_mask=None,
            negative_attention_mask=None,
        )

seconds_start_hidden_states, seconds_end_hidden_states = pipe.encode_duration(
            audio_start_in_s,
            audio_end_in_s,
            device,
            do_classifier_free_guidance and (negative_prompt is not None or negative_prompt_embeds is not None),
            1,
        )

text_audio_duration_embeds = torch.cat(
            [prompt_embeds, seconds_start_hidden_states, seconds_end_hidden_states], dim=1
        )

In [5]:
prompt_embeds.mean(), prompt_embeds.std(), prompt_embeds.shape, text_audio_duration_embeds.shape

(tensor(-0.0002, device='cuda:1', dtype=torch.float16, grad_fn=<MeanBackward0>),
 tensor(0.0634, device='cuda:1', dtype=torch.float16, grad_fn=<StdBackward0>),
 torch.Size([2, 128, 768]),
 torch.Size([2, 130, 768]))

In [6]:
text_audio_duration_embeds.dtype

torch.float16

In [7]:
text_tensor =  np.load('/srv/nfs-data/sisko/matteoc/music/music_data_caps_capt.npy').tolist()
audio_tensor = torch.load('/srv/nfs-data/sisko/matteoc/music/music_data_caps_audio.pt')

/tmp/ipykernel_3107833/2188146880.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  audio_tensor = torch.load('/srv/nfs-data/sisko/matteoc/music/music_data_caps_audio.pt')

In [ ]:
clap_model_id = "laion/larger_clap_music_and_speech"
clap_model = ClapAudioModel.from_pretrained(clap_model_id).to(device)
processor = AutoProcessor.from_pretrained(clap_model_id)


Some weights of ClapAudioModel were not initialized from the model checkpoint at laion/clap-htsat-fused and are newly initialized: ['audio_encoder.batch_norm.bias', 'audio_encoder.batch_norm.num_batches_tracked', 'audio_encoder.batch_norm.running_mean', 'audio_encoder.batch_norm.running_var', 'audio_encoder.batch_norm.weight', 'audio_encoder.layers.0.blocks.0.attention.output.dense.bias', 'audio_encoder.layers.0.blocks.0.attention.output.dense.weight', 'audio_encoder.layers.0.blocks.0.attention.self.key.bias', 'audio_encoder.layers.0.blocks.0.attention.self.key.weight', 'audio_encoder.layers.0.blocks.0.attention.self.query.bias', 'audio_encoder.layers.0.blocks.0.attention.self.query.weight', 'audio_encoder.layers.0.blocks.0.attention.self.relative_position_bias_table', 'audio_encoder.layers.0.blocks.0.attention.self.relative_position_index', 'audio_encoder.layers.0.blocks.0.attention.self.value.bias', 'audio_encoder.layers.0.blocks.0.attention.self.value.weight', 'audio_encoder.layers.

In [18]:
inputs = processor(audios=audio_tensor[0], return_tensors="pt")
outputs = clap_model(**inputs.to(device))
last_hidden_state = outputs.last_hidden_state

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [20]:
outputs.last_hidden_state.shape

torch.Size([1, 768, 2, 32])

In [42]:
text_tensor[1]

'This song features an electric guitar as the main instrument. The guitar plays a descending run in the beginning then plays an arpeggiated chord followed by a double stop hammer on to a higher note and a descending slide followed by a descending chord run. The percussion plays a simple beat using rim shots. The percussion plays in common time. The bass plays only one note on the first count of each bar. The piano plays backing chords. There are no voices in this song. The mood of this song is relaxing. This song can be played in a coffee shop.'

In [51]:
pipe.transformer.config.in_channels, pipe.transformer.config.sample_size

(64, 1024.0)

In [53]:
latents = pipe.prepare_latents(
        batch_size=1*1,
        num_channels_vae=pipe.transformer.config.in_channels,
        sample_size=int(215),   # sample_size=int(pipe.transformer.config.sample_size)
        dtype=torch.float16,
        device=torch.device(device),
        generator=generator,
        latents=None,
        initial_audio_waveforms=None,
        num_waveforms_per_prompt=1,
        audio_channels=pipe.vae.config.audio_channels,
    )

In [54]:
latents.shape

torch.Size([1, 64, 215])

In [43]:
# audio_enc_zeros = torch.zeros_like(audio_tensor[0])
audio_enc = torch.cat((audio_tensor[1].unsqueeze(0), audio_tensor[1].unsqueeze(0)), dim=0)

In [44]:
audio_enc.shape

torch.Size([2, 441000])

In [45]:
enc_output = pipe.vae.encoder(audio_enc.half().to(device).unsqueeze(0))

In [46]:
enc_output.shape

torch.Size([1, 128, 215])

In [47]:
audio = pipe.vae.decode(enc_output[:,0:64,:]).sample
# audio = pipe.vae.decode(enc_output).sample

In [48]:
audio.shape

torch.Size([1, 2, 440320])

In [49]:
waveform_start = int(audio_start_in_s * pipe.vae.config.sampling_rate)
waveform_end = int(audio_end_in_s * pipe.vae.config.sampling_rate)

audio = audio[:, :, waveform_start:waveform_end]

In [50]:
from diffusers import AudioPipelineOutput
from IPython.display import Audio

audio = audio.float().cpu().detach().numpy()

audio_pipe_val = AudioPipelineOutput(audios=audio[0])
Audio(audio_pipe_val[0], rate=44100)